<a href="https://colab.research.google.com/github/Saputoa21/Machine-Translation/blob/main/finetune_FreezemBART_AdvanceMT2025W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece]==4.44.2 # 4.44.2
!pip install datasets==2.19.0
!pip install evaluate==0.4.3
!pip install accelerate==0.34.2
!pip install sacrebleu==2.4.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 80.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstal

In [ ]:
#txt to json
import sys
import json
import re
import codecs

# json from huggingface
#{ "translation": { "en": "Others have dismissed him as a joke.", "ro": "Alții l-au numit o glumă." } }
#{ "translation": { "en": "And some are holding out for an implosion.", "ro": "Iar alții așteaptă implozia." } }

def txt2json(src_id, trg_id, src_file, trg_file, out_file):

  src = codecs.open(src_file, 'r', encoding="utf-8")
  trg = codecs.open(trg_file, 'r', encoding="utf-8")
  out_json = codecs.open(out_file, 'w', encoding="utf-8")


  src_lines = []
  trg_lines = []
  for line_s, line_t in zip(src, trg):
      line_s = line_s.strip()
      line_t = line_t.strip()
      src_lines.append(line_s)
      trg_lines.append(line_t)
  recs = [src_lines, trg_lines]
  for src, tgt in zip(*recs):
      out = {"translation": { src_id: src, trg_id: tgt } }
      x = json.dumps(out, indent=0, ensure_ascii=False)
      x = re.sub(r'\n', ' ', x, 0, re.M)
      out_json.write(x + "\n")
  out_json.close()
  return

In [ ]:
!ls

sample_data				train.en-de.en-filtered.en.semantic.en
train.en-de.de-filtered.de.semantic.de


In [ ]:
#lang_pair = "en-de"

#lang_pair = "en-de"

train_src = "train.en-de.en-filtered.en.semantic.en"
train_trg = "train.en-de.de-filtered.de.semantic.de"
train_json = "train.en-de.json"

txt2json('en_XX', 'de_DE', train_src, train_trg, train_json)

In [ ]:
import numpy as np
import torch
from torch import nn
from typing import Callable, Dict, Iterable, List, Tuple, Union

# Utilities for freezing parameters and checking whether they are frozen
def lmap(f: Callable, x: Iterable) -> List:
    """list(map(f, x))"""
    return list(map(f, x))

def freeze_params(model: nn.Module): #inheritance from Model class
    """Set requires_grad=False for each of model.parameters()"""
    for par in model.parameters(): #for each parameter in a model
        par.requires_grad = False #freezing


def freeze_embeds(model):
    """Freeze token embeddings and positional embeddings for bart, just token embeddings for t5."""
    model_type = model.config.model_type #to learn the type of the model

    if model_type == "t5": #it has only tokens
        freeze_params(model.shared)
        for d in [model.encoder, model.decoder]:
            freeze_params(d.embed_tokens)
    elif model_type == "fsmt":
        for d in [model.model.encoder, model.model.decoder]:
            freeze_params(d.embed_positions)
            freeze_params(d.embed_tokens)
    else:
        freeze_params(model.model.shared)  #mbart is here
        for d in [model.model.encoder, model.model.decoder]:
            freeze_params(d.embed_positions) #positional embeddings
            freeze_params(d.embed_tokens) #token for the word


def grad_status(model: nn.Module) -> Iterable:
    return (par.requires_grad for par in model.parameters())


def any_requires_grad(model: nn.Module) -> bool:
    return any(grad_status(model))


def assert_all_frozen(model): #to check if the model works
    model_grads: List[bool] = list(grad_status(model))
    n_require_grad = sum(lmap(int, model_grads))
    npars = len(model_grads)
    assert not any(model_grads), f"{n_require_grad/npars:.1%} of {npars} weights require grad"


def assert_not_all_frozen(model):
    model_grads: List[bool] = list(grad_status(model))
    npars = len(model_grads)
    assert any(model_grads), f"none of {npars} weights require grad"

In [ ]:
import torch
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    MBartTokenizer,
    MBart50Tokenizer,
    MBartTokenizerFast,
    MBart50TokenizerFast,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from datasets import load_dataset, load_metric, DatasetDict
import transformers
import os
import numpy as np

MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)
MULTILINGUAL_TOKENIZERS = [MBartTokenizer, MBartTokenizerFast, MBart50Tokenizer, MBart50TokenizerFast]


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel() #to count parameters that are not frozen
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

def main():

    model_id = "facebook/mbart-large-50-one-to-many-mmt"
    max_length = 100
    freeze_embed = True
    freeze_encoder = True

    code2lang = {
    "de": "German",
    "fr": "French",
    "en": "English",
    "nl": "Dutch",
    "pt": "Portuguese",
    "ru": "Russian",
    "zh": "Chinese",
    "ro": "Romanian",
    "es": "Spanish"
    }
    source_code = 'en_XX'
    target_code = 'de_DE'
    forced_bos_token = 'de_DE' #TODO!!!??? or empty str""
    data_files = "train.en-de.json"#
    #https://arxiv.org/pdf/2312.12740.pdf trainig size 20k
    output_dir = 'models/mbart50-finetune_freeze_emb_enc'
    train_bs = 6 #bigger batch size
    grad_acc = 4 #24 sentences in the batch
    lr = 1e-4
    w_steps = 0.03
    n_epoch = 2
    lr_scheduler_type = "linear" #'cosine', etc
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id,
                                                  device_map="auto") #{"": 0} #to map to ressources

    model.resize_token_embeddings(len(tokenizer))

    # Set decoder_start_token_id
    if model.config.decoder_start_token_id is None and isinstance(tokenizer, (MBartTokenizer, MBartTokenizerFast)):
      if isinstance(tokenizer, MBartTokenizer, MBartTokenizerFast):
          model.config.decoder_start_token_id = tokenizer.lang_code_to_id[target_code] #Ru
      else:
          model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(target_code)


    if isinstance(tokenizer, tuple(MULTILINGUAL_TOKENIZERS)):

      tokenizer.src_lang = source_code
      tokenizer.tgt_lang = target_code

      # For multilingual translation models like mBART-50 and M2M100 we need to force the target language token
      # as the first generated token.
      forced_bos_token_id = (
          tokenizer.lang_code_to_id[forced_bos_token] if forced_bos_token is not None else None
      )
      model.config.forced_bos_token_id = forced_bos_token_id
      #print(tokenizer.src_lang, model.config.forced_bos_token_id)


  # do 3 versions (embed only, param only, both)
    if freeze_embed:
      print("***** freeze embeddings *****")
      freeze_embeds(model)

    if freeze_encoder:
      print("***** freeze encoder *****")
      freeze_params(model.get_encoder())
      assert_all_frozen(model.get_encoder())

    print_trainable_parameters(model)
    print(model)

    metric = load_metric('sacrebleu', trust_remote_code=True)

    def preprocess_parallel_function(examples):
      inputs = [ex[source_code] for ex in examples["translation"]]
      targets = [ex[target_code] for ex in examples["translation"]]
      #inputs = [prefix + inp for inp in inputs]
      #print(inputs, targets)
      model_inputs = tokenizer(inputs, max_length=max_length, padding=False, truncation=True)


      labels = tokenizer(targets, max_length=max_length, padding=False, truncation=True)

      #if padding == "max_length" and ignore_pad_token_for_loss:
      #labels["input_ids"] = [
      #    [(l if l != tokenizer.pad_token_id else -100) for l in label]
      #    for label in labels["input_ids"]]



      model_inputs["labels"] = labels["input_ids"]

      return model_inputs

    def postprocess_text(preds, labels):
      preds = [pred.strip() for pred in preds]
      labels = [[label.strip()] for label in labels]

      return preds, labels

    def compute_metrics(eval_preds, ignore_pad_token_for_loss=False):
      preds, labels = eval_preds
      if isinstance(preds, tuple):
        preds = preds[0]
      decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
      # Replace -100 in the labels as we can't decode them.
      labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
      decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
      # Some simple post-processing
      decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

      result = metric.compute(predictions=decoded_preds, references=decoded_labels)
      prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
      result = {'bleu' : result['score']}
      result["gen_len"] = np.mean(prediction_lens)
      result = {k: round(v, 4) for k, v in result.items()}
      return result


    #data = load_from_disk("path_directory")
    #put data_save_folder from drive
    data = load_dataset("json", data_files=data_files)
    data_split_tmp = data['train'].train_test_split(test_size=0.2, seed=42)
    data_split_valid_test = data_split_tmp['test'].train_test_split(test_size=0.5, seed=42)
    data = DatasetDict({'train': data_split_tmp['train'].select(range(0, 5000)), #check your credits!!!
                        'valid': data_split_valid_test['train'].select(range(0, 200)),
                        'test':  data_split_valid_test['test'].select(range(0, 200)),})
    #data['test']
    column_names = data["train"].column_names
    #print(column_names)
    data = data.map(preprocess_parallel_function,
                    batched=True)
    label_pad_token_id = -100 #TODO??

    trainer = transformers.Seq2SeqTrainer(
        model=model,
        train_dataset=data["train"],
        eval_dataset=data["valid"],
        args=transformers.Seq2SeqTrainingArguments(
            report_to='none', #turn off  wandb
            per_device_train_batch_size=train_bs, #4, 12
            gradient_accumulation_steps=grad_acc,
            do_train=True,
            do_eval=True,
            per_device_eval_batch_size=4,
            eval_accumulation_steps=2,
            warmup_ratio=w_steps,
            lr_scheduler_type=lr_scheduler_type,
            num_train_epochs=n_epoch, #5?
            predict_with_generate=True,
            metric_for_best_model='bleu',
            load_best_model_at_end=True,
            learning_rate=lr, #0.1, 0.01, 0.001
            save_total_limit=1,
            generation_num_beams=4,
            save_strategy="epoch",
            eval_strategy="epoch",
            output_dir=output_dir,
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(tokenizer,
                                                          label_pad_token_id=label_pad_token_id,
                                                          model=model),
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
    trainer.train()

    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)


In [ ]:
#fine-tune
main()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


***** freeze embeddings *****
***** freeze encoder *****
trainable params: 201564160 || all params: 610879488 || trainable%: 32.99573221224282
MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
         

/tmp/ipython-input-2109429943.py:114: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('sacrebleu', trust_remote_code=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
0,No log,0.885356,41.288600,32.465000
1,No log,0.856925,42.677100,31.655000


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250003, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig fi

In [ ]:
#!cp -r models/mbart50_finetune drive/

In [ ]:
!pwd

In [ ]:
!tar -czvf mbart50-finetune.tar.gz models/mbart50-finietune

In [ ]:
import gc
#del(model)
gc.collect()
torch.cuda.empty_cache()

# Freezed Fine-Tuning with my Corpus

....

## Only Embeddings

In [ ]:
import numpy as np
import torch
from torch import nn
from typing import Callable, Dict, Iterable, List, Tuple, Union

# Utilities for freezing parameters and checking whether they are frozen
def lmap(f: Callable, x: Iterable) -> List:
    """list(map(f, x))"""
    return list(map(f, x))

def freeze_params(model: nn.Module): #inheritance from Model class
    """Set requires_grad=False for each of model.parameters()"""
    for par in model.parameters(): #for each parameter in a model
        par.requires_grad = False #freezing


def freeze_embeds(model):
    """Freeze token embeddings and positional embeddings for bart, just token embeddings for t5."""
    model_type = model.config.model_type #to learn the type of the model

    if model_type == "t5": #it has only tokens
        freeze_params(model.shared)
        for d in [model.encoder, model.decoder]:
            freeze_params(d.embed_tokens)
    elif model_type == "fsmt":
        for d in [model.model.encoder, model.model.decoder]:
            freeze_params(d.embed_positions)
            freeze_params(d.embed_tokens)
    else:
        freeze_params(model.model.shared)  #mbart is here
        for d in [model.model.encoder, model.model.decoder]:
            freeze_params(d.embed_positions) #positional embeddings
            freeze_params(d.embed_tokens) #token for the word


def grad_status(model: nn.Module) -> Iterable:
    return (par.requires_grad for par in model.parameters())


def any_requires_grad(model: nn.Module) -> bool:
    return any(grad_status(model))


def assert_all_frozen(model): #to check if the model works
    model_grads: List[bool] = list(grad_status(model))
    n_require_grad = sum(lmap(int, model_grads))
    npars = len(model_grads)
    assert not any(model_grads), f"{n_require_grad/npars:.1%} of {npars} weights require grad"


def assert_not_all_frozen(model):
    model_grads: List[bool] = list(grad_status(model))
    npars = len(model_grads)
    assert any(model_grads), f"none of {npars} weights require grad"

In [ ]:
import torch
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    MBartTokenizer,
    MBart50Tokenizer,
    MBartTokenizerFast,
    MBart50TokenizerFast,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from datasets import load_dataset, load_metric, DatasetDict
import transformers
import os
import numpy as np

MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)
MULTILINGUAL_TOKENIZERS = [MBartTokenizer, MBartTokenizerFast, MBart50Tokenizer, MBart50TokenizerFast]


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel() #to count parameters that are not frozen
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

def main():

    model_id = "facebook/mbart-large-50-one-to-many-mmt"
    max_length = 100
    freeze_embed = True
    freeze_encoder = True

    code2lang = {
    "en": "English",
    "ru": "Russian",
    }

    source_code = 'en_XX'
    target_code = 'ru_RU'
    forced_bos_token = 'ru_RU'
    data_files = "train.en-ru.json" #https://arxiv.org/pdf/2312.12740.pdf trainig size 20k
    output_dir = 'models/mbart50-finetune_freeze_emb_enc'
    train_bs = 6 #bigger batch size
    grad_acc = 4 #24 sentences in the batch
    lr = 1e-4
    w_steps = 0.03
    n_epoch = 2
    lr_scheduler_type = "linear" #'cosine', etc
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id,
                                                  device_map="auto") #{"": 0} #to map to ressources

    model.resize_token_embeddings(len(tokenizer))

    # Set decoder_start_token_id
    if model.config.decoder_start_token_id is None and isinstance(tokenizer, (MBartTokenizer, MBartTokenizerFast)):
      if isinstance(tokenizer, MBartTokenizer, MBartTokenizerFast):
          model.config.decoder_start_token_id = tokenizer.lang_code_to_id[target_code] #Ru
      else:
          model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(target_code)


    if isinstance(tokenizer, tuple(MULTILINGUAL_TOKENIZERS)):

      tokenizer.src_lang = source_code
      tokenizer.tgt_lang = target_code

      # For multilingual translation models like mBART-50 and M2M100 we need to force the target language token
      # as the first generated token.
      forced_bos_token_id = (
          tokenizer.lang_code_to_id[forced_bos_token] if forced_bos_token is not None else None
      )
      model.config.forced_bos_token_id = forced_bos_token_id
      #print(tokenizer.src_lang, model.config.forced_bos_token_id)


  # do 3 versions (embed only, param only, both)
    if freeze_embed:
      print("***** freeze embeddings *****")
      freeze_embeds(model)

    if freeze_encoder:
      print("***** freeze encoder *****")
      freeze_params(model.get_encoder())
      assert_all_frozen(model.get_encoder())

    print_trainable_parameters(model)
    print(model)

    metric = load_metric('sacrebleu', trust_remote_code=True)

    def preprocess_parallel_function(examples):
      inputs = [ex[source_code] for ex in examples["translation"]]
      targets = [ex[target_code] for ex in examples["translation"]]
      #inputs = [prefix + inp for inp in inputs]
      #print(inputs, targets)
      model_inputs = tokenizer(inputs, max_length=max_length, padding=False, truncation=True)


      labels = tokenizer(targets, max_length=max_length, padding=False, truncation=True)

      #if padding == "max_length" and ignore_pad_token_for_loss:
      #labels["input_ids"] = [
      #    [(l if l != tokenizer.pad_token_id else -100) for l in label]
      #    for label in labels["input_ids"]]


      model_inputs["labels"] = labels["input_ids"]

      return model_inputs

    def postprocess_text(preds, labels):
      preds = [pred.strip() for pred in preds]
      labels = [[label.strip()] for label in labels]

      return preds, labels

    def compute_metrics(eval_preds, ignore_pad_token_for_loss=False):
      preds, labels = eval_preds
      if isinstance(preds, tuple):
        preds = preds[0]
      decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
      # Replace -100 in the labels as we can't decode them.
      labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
      decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
      # Some simple post-processing
      decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

      result = metric.compute(predictions=decoded_preds, references=decoded_labels)
      prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
      result = {'bleu' : result['score']}
      result["gen_len"] = np.mean(prediction_lens)
      result = {k: round(v, 4) for k, v in result.items()}
      return result


    #data = load_from_disk("path_directory")
    #put data_save_folder from drive
    data = load_dataset("json", data_files=data_files)
    data_split_tmp = data['train'].train_test_split(test_size=0.2, seed=42)
    data_split_valid_test = data_split_tmp['test'].train_test_split(test_size=0.5, seed=42)
    data = DatasetDict({'train': data_split_tmp['train'].select(range(0, 5000)), #check your credits!!!
                        'valid': data_split_valid_test['train'].select(range(0, 200)),
                        'test':  data_split_valid_test['test'].select(range(0, 200)),})
    #data['test']
    column_names = data["train"].column_names
    #print(column_names)
    data = data.map(preprocess_parallel_function,
                    batched=True)
    label_pad_token_id = -100

    trainer = transformers.Seq2SeqTrainer(
        model=model,
        train_dataset=data["train"],
        eval_dataset=data["valid"],
        args=transformers.Seq2SeqTrainingArguments(
            report_to='none', #turn off  wandb
            per_device_train_batch_size=train_bs, #4, 12
            gradient_accumulation_steps=grad_acc,
            do_train=True,
            do_eval=True,
            per_device_eval_batch_size=4,
            eval_accumulation_steps=2,
            warmup_ratio=w_steps,
            lr_scheduler_type=lr_scheduler_type,
            num_train_epochs=n_epoch, #5?
            predict_with_generate=True,
            metric_for_best_model='bleu',
            load_best_model_at_end=True,
            learning_rate=lr, #0.1, 0.01, 0.001
            save_total_limit=1,
            generation_num_beams=4,
            save_strategy="epoch",
            eval_strategy="epoch",
            output_dir=output_dir,
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(tokenizer,
                                                          label_pad_token_id=label_pad_token_id,
                                                          model=model),
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
    trainer.train()

    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

In [ ]:
#fine-tune
main(freeze_embed = True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


***** freeze embeddings *****
***** freeze encoder *****
trainable params: 201564160 || all params: 610879488 || trainable%: 32.99573221224282
MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
         

/tmp/ipython-input-2109429943.py:114: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('sacrebleu', trust_remote_code=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
0,No log,0.885356,41.288600,32.465000
1,No log,0.856925,42.677100,31.655000


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250003, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig fi

In [ ]:
#use models
#import models and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_from_disk
tokenizer = AutoTokenizer.from_pretrained("models/mbart50-full-finetune-1epoch-1e4") #path of your directory with the finetuned

model = AutoModelForSeq2SeqLM.from_pretrained("models/mbart50-full-finetune-1epoch-1e4") #gpu!!!

#upload source file and read
import codecs
#file_name = "Vienna_Environmental.en-de.valid.en"
mt_output = []
output = open('mtouput_fullFT', 'w', encoding='utf-8')
data = load_from_disk("data_save_test")
print(data)
data['test']
#with codecs.open(file_name, 'r', 'utf-8') as src:
for line in data['test']: #trasn src and trg translation src and trg
    src = line['translation']['en_XX']
    trg = line['translation']['ru_RU']
    #line = line.strip()
    encoded = tokenizer(src, return_tensors="pt")              #gpu?? tokenize
    generated_tokens = model.generate(**encoded,
                                      forced_bos_token_id=tokenizer.lang_code_to_id['ru_RU']) #add beam search 4
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True) #de tokenize
    print(translation[0], file=output)
    print(translation[0])

## Only encoder

In [ ]:
#fine-tune
main(freeze_encoder = True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


***** freeze embeddings *****
***** freeze encoder *****
trainable params: 201564160 || all params: 610879488 || trainable%: 32.99573221224282
MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
         

/tmp/ipython-input-2109429943.py:114: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('sacrebleu', trust_remote_code=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
0,No log,0.885356,41.288600,32.465000
1,No log,0.856925,42.677100,31.655000


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250003, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig fi

In [ ]:
#use models
#import models and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_from_disk
tokenizer = AutoTokenizer.from_pretrained("models/mbart50-full-finetune-1epoch-1e4") #path of your directory with the finetuned

model = AutoModelForSeq2SeqLM.from_pretrained("models/mbart50-full-finetune-1epoch-1e4") #gpu!!!

#upload source file and read
import codecs
#file_name = "Vienna_Environmental.en-de.valid.en"
mt_output = []
output = open('mtouput_fullFT', 'w', encoding='utf-8')
data = load_from_disk("data_save_test")
print(data)
data['test']
#with codecs.open(file_name, 'r', 'utf-8') as src:
for line in data['test']: #trasn src and trg translation src and trg
    src = line['translation']['en_XX']
    trg = line['translation']['ru_RU']
    #line = line.strip()
    encoded = tokenizer(src, return_tensors="pt")              #gpu?? tokenize
    generated_tokens = model.generate(**encoded,
                                      forced_bos_token_id=tokenizer.lang_code_to_id['ru_RU']) #add beam search 4
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True) #de tokenize
    print(translation[0], file=output)
    print(translation[0])

## Both

In [ ]:
#fine-tune
main(freeze_encoder = True, freeze_embed = True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


***** freeze embeddings *****
***** freeze encoder *****
trainable params: 201564160 || all params: 610879488 || trainable%: 32.99573221224282
MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
         

/tmp/ipython-input-2109429943.py:114: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('sacrebleu', trust_remote_code=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
0,No log,0.885356,41.288600,32.465000
1,No log,0.856925,42.677100,31.655000


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250003, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig fi

In [ ]:
#use models
#import models and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_from_disk
tokenizer = AutoTokenizer.from_pretrained("models/mbart50-full-finetune-1epoch-1e4") #path of your directory with the finetuned

model = AutoModelForSeq2SeqLM.from_pretrained("models/mbart50-full-finetune-1epoch-1e4") #gpu!!!

#upload source file and read
import codecs
#file_name = "Vienna_Environmental.en-de.valid.en"
mt_output = []
output = open('mtouput_fullFT', 'w', encoding='utf-8')
data = load_from_disk("data_save_test")
print(data)
data['test']
#with codecs.open(file_name, 'r', 'utf-8') as src:
for line in data['test']: #trasn src and trg translation src and trg
    src = line['translation']['en_XX']
    trg = line['translation']['ru_RU']
    #line = line.strip()
    encoded = tokenizer(src, return_tensors="pt")              #gpu?? tokenize
    generated_tokens = model.generate(**encoded,
                                      forced_bos_token_id=tokenizer.lang_code_to_id['ru_RU']) #add beam search 4
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True) #de tokenize
    print(translation[0], file=output)
    print(translation[0])